# Organize data into excel workbooks

2024.2.16

2024.5.14 V2

In [1]:
import time

print(time.asctime())

import os
from pathlib import Path

import pandas as pd
import numpy as np
import json
from scipy.io import loadmat

FIGURE_DATA_PATH = Path("../data")
os.chdir(FIGURE_DATA_PATH)

np.set_printoptions(precision=5)
SRCDATA_PATH = Path("../sourcedata")
def format_float_list(x, precision=6):
    return "[" + ",".join([f"%.{precision}g" % i for i in x]) + "]"
def format_int_list(x):
    return "[" + ",".join([f"{i:d}" for i in x]) + "]"
def check_closed(x:str):
    return (x[-1]==']') and (len(x)<32768)

Mon May 20 15:39:58 2024


## Figure 1

### 1l

In [25]:
fig1kl = pd.read_excel("fig1/fig1kl.xlsx")

In [20]:
fig1kl.to_excel(SRCDATA_PATH / "figure_1.xlsx", index=False, sheet_name='fig1l')

## Figure 2

### 2d

In [123]:
with open("fig2/fig2d.json", "r") as fp:
    fig2d = json.load(fp)

In [124]:
fig2d_df = pd.DataFrame(fig2d, index=[f"Rat {i}" for i in range(1, 6)])

In [125]:
for k in ["amplitudes", "snrs", "frs"]:
    fig2d_df[k] = fig2d_df[k].apply(format_float_list)

for k in ["densities", "spreads"]:
    fig2d_df[k] = fig2d_df[k].apply(format_int_list)

In [126]:
fig2d_df=pd.DataFrame(fig2d_df)

In [127]:
for k in ["amplitudes", "snrs", "frs", "densities", "spreads"]:
    print(fig2d_df[k].apply(check_closed).all())

True
True
True
True
True


In [128]:
fig2d_df.rename(columns={
    "efficiencies": "Efficiency",
    "amplitudes": "Amplitude",
    "snrs": "SNR",
    "frs": "Firing Rate",
    "densities": "Density",
    "spreads": "Spread"
}, inplace=True)

In [129]:
fig2d_df.drop(columns=["yields",'yields_good_mua','yields_mua'], inplace=True)

In [130]:
with pd.ExcelWriter(
    SRCDATA_PATH / "figure_2.xlsx",
    mode="w",
    engine="openpyxl",
) as writer:
    fig2d_df.to_excel(writer, sheet_name="fig2d", index=True)

### 2j

In [131]:
with open("fig2/fig2j.json", "r") as fp:
    fig2j = json.load(fp)
fig2j_df = pd.DataFrame(fig2j)
fig2j_df = fig2j_df.T
fig2j_df.index = [f"Rat {i}" for i in range(1, 7)]

In [132]:
# fr_values_dict = {}
# for k in fig2j_df.fr_values.keys():
#     for point, values in enumerate(fig2j_df.fr_values[k]):
#         if k not in fr_values_dict:
#             fr_values_dict["%s-Point %d" % (k, point)] = []
#         fr_values_dict["%s-Point %d" % (k, point)].append(format_float_list(values))
# fr_values_dict=pd.DataFrame(fr_values_dict).T
# fr_values_dict.columns=['Firing rate of each unit']
# fr_values_dict['Firing rate of each unit'].apply(check_closed).all()

True

In [133]:
for k in ["tp", "cluster_count", "channels_have_signals_count"]:
    fig2j_df[k] = fig2j_df[k].apply(format_int_list)
for k in ["tot_fr"]:
    fig2j_df[k] = fig2j_df[k].apply(format_float_list)

In [134]:
fig2j_df['Mean amplitude']=fig2j_df.amp.apply(lambda x:[np.mean(i) for i in x])
fig2j_df['Mean amplitude']=fig2j_df['Mean amplitude'].apply(format_float_list)
fig2j_df.drop(columns=['amp','fr_values'], inplace=True)

In [135]:
for k in fig2j_df.columns:
    print(fig2j_df[k].apply(check_closed).all())

True
True
True
True
True


In [136]:
fig2j_df.rename(columns={
    "tp": "Timepoint (week)",
    "cluster_count": "Single unit count",
    "channels_have_signals_count": "Count of channels with SUA",
    "tot_fr": "Total firing rate",
}, inplace=True)

In [137]:
with pd.ExcelWriter(SRCDATA_PATH / "figure_2.xlsx", mode="a", engine='openpyxl') as writer:
    fig2j_df.to_excel(writer, sheet_name="fig2j", index=True)

In [138]:
# with pd.ExcelWriter(SRCDATA_PATH / "figure_2.xlsx", mode="a", engine='openpyxl') as writer:
#     fr_values_dict.to_excel(writer, sheet_name="fig2j_firingrates", index=True)

In [139]:
fig2j_pvalues = pd.read_excel("fig2/fig2j_pvalues.xlsx")
fig2j_pvalues.columns = map(lambda x: x.replace('nc','Neuron count').replace('fr','Firing rate').replace('amp','Amplitude').replace('active_ch','Count of channels with SUA'), fig2j_pvalues.columns)
fig2j_pvalues.drop(columns=['rat'], inplace=True)
with pd.ExcelWriter(SRCDATA_PATH / "figure_2.xlsx", mode="a", engine='openpyxl') as writer:
    fig2j_pvalues.to_excel(writer, sheet_name="fig2j_regression_results", index=False)

### 2g/ext6c

In [140]:
fig2g = pd.read_excel("fig2/fig2g.xlsx")
fig2g.ksd_good.apply(check_closed).all()
fig2g.ksd_good = fig2g.ksd_good.apply(eval).apply(lambda x: format_float_list(x))

In [141]:
fig2g = pd.read_excel("fig2/fig2g.xlsx")
fig2g.ksd_good.apply(check_closed).all()
fig2g.ksd_good = fig2g.ksd_good.apply(eval).apply(lambda x: format_float_list(x))
fig2g.rename(columns={"timepoint_int":"Timepoint (week)"}, inplace=True)
fig2g=fig2g[['Timepoint (week)','Amplitude']]
with pd.ExcelWriter(
    SRCDATA_PATH / "figure_2.xlsx", mode="a", engine="openpyxl"
) as writer:
    fig2g.to_excel(writer, sheet_name="fig2g", index=False)

---

In [142]:
fig2g = pd.read_excel("ext/ext6c.xlsx")
fig2g.ksd_good.apply(check_closed).all()
fig2g.ksd_good = fig2g.ksd_good.apply(eval).apply(lambda x: format_float_list(x))
fig2g.rename(columns={"timepoint_int":"Timepoint (week)"}, inplace=True)
fig2g=fig2g[['Timepoint (week)','Amplitude']]
with pd.ExcelWriter(
    SRCDATA_PATH / "ext_figure_6.xlsx", mode="w", engine="openpyxl"
) as writer:
    fig2g.to_excel(writer, sheet_name="c", index=False)

### 2h/ext6d

In [143]:
fig2h = pd.read_json("fig2/fig2h.json")
fig2h.snr = fig2h.snr.apply(format_float_list)
fig2h.snr.apply(check_closed).all()
fig2h.rename(columns={"snr": "SNR","timepoint_int":"Timepoint (week)"}, inplace=True)
with pd.ExcelWriter(
    SRCDATA_PATH / "figure_2.xlsx", mode="a", engine="openpyxl"
) as writer:
    fig2h.to_excel(writer, sheet_name="fig2h", index=False)

---

In [144]:
fig2h = pd.read_json("ext/ext6d.json")
fig2h.snr = fig2h.snr.apply(format_float_list)
fig2h.snr.apply(check_closed).all()
fig2h.rename(columns={"snr": "SNR","timepoint_int":"Timepoint (week)"}, inplace=True)
with pd.ExcelWriter(
    SRCDATA_PATH / "ext_figure_6.xlsx", mode="a", engine="openpyxl"
) as writer:
    fig2h.to_excel(writer, sheet_name="d", index=False)

## Figure 3

### 3d

In [225]:
fig3d = pd.read_excel("fig3/fig3d.xlsx")

In [226]:
with pd.ExcelWriter(
    SRCDATA_PATH / "figure_3.xlsx", mode="w", engine="openpyxl"
) as writer:
    fig3d[['channel_id','density']].to_excel(writer, sheet_name="fig3d", index=False)

### 3h/4f

In [227]:
with open("fig3/fig3h.json", "r") as fp:
    fig3h = json.load(fp)
with open("fig4/fig4f.json", "r") as fp:
    fig4f = json.load(fp)

In [228]:
fig3h.pop("dist_division")
fig4f.pop("dist_division")

[0.0, 1.4409999999999998, 10.241, 14.041, 16.641000000000002, 20.841]

In [229]:
fig3h=pd.DataFrame(fig3h)
fig4f=pd.DataFrame(fig4f)

In [230]:
fig3h.drop(columns=["intervals","yield"], inplace=True)
fig4f.drop(columns=["intervals","yield"], inplace=True)

In [231]:
fig3h.rename(
    columns={
        "density": "Density",
        "efficiency": "Efficiency",
        "amplitude": "Amplitude",
        "area_name": "Area",
        "spread": "Spread",
    },
    inplace=True,
)

In [232]:
fig4f.rename(
    columns={
        "density": "Density",
        "efficiency": "Efficiency",
        "amplitude": "Amplitude",
        "area_name": "Area",
        "spread": "Spread",
    },
    inplace=True,
)

In [233]:
with pd.ExcelWriter(
    SRCDATA_PATH / "figure_3.xlsx", mode="a", engine="openpyxl"
) as writer:
    pd.DataFrame(fig3h).to_excel(writer, sheet_name="fig3h", index=False)

In [234]:
with pd.ExcelWriter(
    SRCDATA_PATH / "figure_4.xlsx", mode="w", engine="openpyxl"
) as writer:
    pd.DataFrame(fig4f).to_excel(writer, sheet_name="fig4f", index=False)

### 3i

In [235]:
fig3i = pd.read_json("fig3/fig3i.json")

In [236]:
fig3i.drop(columns="efficiencies",inplace=True)

In [237]:
for k in ["amplitudes", "firingrates", "snr"]:
    fig3i[k] = fig3i[k].apply(format_float_list)
    print(fig3i[k].apply(check_closed).all())

True
True
True


In [238]:
fig3i.rename(columns={
    "probe": "Probe",
    "area_names": "Area",
    "amplitudes": "Amplitude",
    "snr": "SNR",
    "firingrates": "Firing Rate",
    "yields":"Single Unit Count (Yield)"
}, inplace=True)

In [239]:
with pd.ExcelWriter(
    SRCDATA_PATH / "figure_3.xlsx", mode="a", engine="openpyxl", if_sheet_exists='replace'
) as writer:
    fig3i.to_excel(writer, sheet_name="fig3i", index=False)

## Figure 4

### 4d

In [240]:
Probe2_cluster_metrics = pd.read_excel("fig4/fig4d.xlsx")

In [242]:
Probe2_cluster_metrics.rename(
    columns={
        "mch":'channel_id',
        'real_amp':'Amplitude',
        'waveform_type':'Waveform type',
    },
    inplace=True,
)

## Figure 5

### 5e

In [11]:
with open("fig5/fig5e_lower.json") as fp:
    fig5el_data = json.load(fp)

In [12]:
fig5el_data=pd.DataFrame(fig5el_data)

In [13]:
for k in ["error", "swfr_mean"]:
    fig5el_data[k] = fig5el_data[k].apply(format_float_list)
    print(fig5el_data[k].apply(check_closed).all())

True
True


In [14]:
fig5el_data.rename(
    columns={
        "group": "Category",
        "trial_ids": "Trial IDs",
        "swtimes": "Time",
        "swfrs": "Firing Rate",
        "error": "Error",
        "swfr_mean": "Mean firing rate",
    },
    inplace=True
)

In [15]:
with pd.ExcelWriter(
    SRCDATA_PATH / "figure_5.xlsx", mode="w", engine="openpyxl"
) as writer:
    fig5el_data[['Category','Trial IDs','Time','Mean firing rate','Error']].to_excel(writer, sheet_name="fig5e_lower", index=False)

In [ ]:
# fig5el_by_trial = []

# for n, (g, trial_ids, _, swfrs, _, _) in pd.DataFrame(
#     fig5el_data
# ).iterrows():
    

#     for trial_id, swfr in zip(trial_ids,swfrs):

#         fig5el_by_trial.append(
#             {
#                 "Category": g,
#                 "Trial ID": trial_id,
#                 "Firing Rates": swfr,
#             }
#         )
# fig5el_by_trial = pd.DataFrame(fig5el_by_trial)
# fig5el_by_trial["Firing Rates"].apply(str).apply(check_closed).all()

In [20]:
# with pd.ExcelWriter(
#     SRCDATA_PATH / "figure_5.xlsx", mode="a", engine="openpyxl"
# ) as writer:
#     fig5el_by_trial.to_excel(writer, sheet_name="fig5e_lower_firingrates", index=False)